<a href="https://colab.research.google.com/github/adarevalin/Modelos_Lenguaje_Natural/blob/main/CLASIFICADOR_SPAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CLASIFICADOR DE SPAM

Estas son las librerias para importar y realizar todo los procesamientos necesarios.

In [72]:
import nltk, random # necesaria para tokenizar y random nos permite generar aletoriedad en los datos.
nltk.download('punkt')  # Descarga de recursos para tokenización, ejecutar solo una vez

from nltk import word_tokenize  # Tokenización de palabras

from nltk.corpus import stopwords
nltk.download('stopwords')  # Descarga de recursos para palabras vacías, ejecutar solo una vez
stop_words = set(stopwords.words('english'))

import pandas as pd  # Para manipulación de datos
import numpy as np  # Para operaciones numéricas
import zipfile  # Para trabajar con archivos zip

#from google.colab import drive  # Para montar Google Drive
#drive.mount('/content/drive')  # Montar Google Drive para acceder a los archivos


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#1. Cargamos el dataset

Pre - procesamiento de los datos para obtener los atributos necesarios.

* En este caso vamos a sacar 5 bigramas con la mayor distribución de frecuencia que se encuentran en los correos tanto de spam y ham.

1. Cargamos los corpus que vamos a utilizar, en este caso tenemos un dataset que contiene correos en ingles de ham y spam, son de uso libre.

In [73]:
# Carga de los corpus
!git clone https://github.com/pachocamacho1990/datasets

fatal: destination path 'datasets' already exists and is not an empty directory.


2. Procesamos los datos y cargarlos, vamos a utilizar el corpus 1,2 y 3

In [74]:
# Estas expresiones regulares me ayudan hacer busquedas y filtrar las palabras importantes.

pattern = r'''(?x)                 # establecer bandera para permitir expresiones regulares detalladas
              (?:[A-Z]\.)+         # abreviaturas, ejemplo (U.S.A.)
              | \w+(?:-\w+)*       # palabras con guiones internos opcionales
              | \$?\d+(?:\.\d+)?%? # moneda y porcentajes, ejemplo ($12.40, 82%)
              | \.\.\.             # elipsis
              | [][.,;"'?():-_`]   # éstas son fichas separadas; incluye ], [
'''

In [75]:
from nltk.util import bigrams # importar de nltk los bigrams

ruta = "/content/datasets/email/plaintext/corpus2.zip" # este es el corpus 1 de correos en inglish

data = [] # donde vamos a guardar los bigrams

with zipfile.ZipFile(ruta, mode="r") as corpus: # el with es necesario para cerrar todos los ficheros luego de terminar el proceso
    for file in corpus.namelist():
        with corpus.open(file) as f:
            text = f.read().decode('ISO-8859-1')  # Decodificar el texto
            if text.strip():  # Verificar si el texto no está vacío

                tokens = nltk.regexp_tokenize(text,pattern) # tokenizar y realizar filtro con expresión regulares

                long_words = [palabra for palabra in tokens if palabra.lower() not in stop_words and len(palabra) > 2] # aqui estamos sacando las palabras que no estan en stopwords y ademas con longitud mayor a dos

                bigramas = list(bigrams(long_words)) # usando la clase bigrams obtenemos las posibles combinaciones

                bandera = 3
                fil_bigramas = [bigrams for bigrams in bigramas if len(bigrams[0]) > bandera and len(bigrams[1]) > bandera] # realizamos un flitro para las palabras de los bigramas
                fdistBigrams = nltk.FreqDist(fil_bigramas) # obtenemos la distribucion de frecuencia
                top_bigrams = fdistBigrams.most_common(5) # guardamos los 5 mayores bigramas, que serian nuestras colocaciones.

                # Etiquetar el archivo según su clase (ham o spam)
                label = 'True' if 'ham' in file else 'False' # si ham esta en los archivos es true sino es false

                data.append((top_bigrams,label)) # agregamos los datos y sus etiquetas a la lista de data.
                random.shuffle(data) # con la funcion random creamos aletoriedad.


En df obtenemos las colocaciones y su frecuencia en el texto, tener presente que el analisis se hizo para cada correo, por lo cual estas colocaciones y frecuencias son propias de cada uno.

In [76]:
df = pd.DataFrame(data, columns=['top_bigrams','Clase']) # convertimos con pandas un dataframe
df.head(10) # graficamos los primeros 10 valores.

,top_bigrams,Clase
0,"[((merchant, asset), 4), ((asset, group), 4), ...",True
1,"[((Subject, mail), 1), ((mail, delivery), 1), ...",False
2,"[((sept, confirmation), 2), ((Subject, wellesl...",True
3,"[((Subject, entrust), 1), ((entrust, visual), ...",False
4,"[((issue, issue), 27), ((2005, issue), 8), ((a...",False
5,"[((floor, meetings), 4), ((floor, trading), 4)...",True
6,"[((Subject, hassle), 1), ((hassle, free), 1), ...",False
7,"[((continental, power), 4), ((experience, brea...",True
8,"[((sales, marketing), 3), ((relevant, sales), ...",False
9,"[((july, 2000), 2), ((Subject, staff), 1), ((s...",True


ya que para entrenar el modelo de naivesbayes se debe pasarle una lista de clave:valor entonces se crea un diccionario.

In [151]:

#Si el bigrama está presente en el documento, el valor asociado en el diccionario de atributos será 'True', de lo contrario será 'False'.
def documento_atributo_bigram(top_bigrams):

    atrib = {} # diccionario de atributos

    for bigrama in top_bigrams: # obtenemos los bigramas de la lista.

        atrib[f"contains({bigrama})"] = bigrama # agregamos las colocaciones con la estructura clave:valor

    return atrib # retornamos el diccionario



# Vamos a preparar el dataset

Ya que tenemos mas de un corpus vamos a utilizar el corpus 1 solo para entrenar los datos y los restantes de prueba, con el fin de ver de manera precisa las metricas de validación.

In [78]:
#Tomando los bigramas mas repetidos por frecuencia

fset_corpus_bigram_freq = [(documento_atributo_bigram(top_bigrams),
         clase) for top_bigrams,clase in zip(df["top_bigrams"],df["Clase"])] # aqui utilizamos la funcion para crear el diccionario.

random.shuffle(fset_corpus_bigram_freq) # volvemos aplicar random
train = fset_corpus_bigram_freq[:]


In [79]:
train[:2]

[({"contains((('Subject', 'software'), 1))": (('Subject', 'software'), 1),
   "contains((('rush', 'undr'), 1))": (('rush', 'undr'), 1),
   "contains((('undr', 'software'), 1))": (('undr', 'software'), 1),
   "contains((('software', 'paradise'), 1))": (('software', 'paradise'), 1),
   "contains((('paradise', 'honest'), 1))": (('paradise', 'honest'), 1)},
  'False'),
 ({"contains((('Subject', 'viagra'), 1))": (('Subject', 'viagra'), 1),
   "contains((('viagra', 'helps'), 1))": (('viagra', 'helps'), 1),
   "contains((('helps', 'great'), 1))": (('helps', 'great'), 1),
   "contains((('help', 'suspicious'), 1))": (('help', 'suspicious'), 1),
   "contains((('suspicious', 'mind'), 1))": (('suspicious', 'mind'), 1)},
  'False')]

## Vamos a entrenar el modelo

Vamos a utilizar NaiveBayes que utiliza la logica de probabilidades de la regla de bayes.

In [80]:
from nltk.classify import NaiveBayesClassifier # traemos el modelo de nltk
from nltk.classify import accuracy as ac
from nltk.metrics.scores import recall, precision, accuracy

In [81]:
clasif = NaiveBayesClassifier.train(train) # creamos la intancia y entrenamos el modelo con train

## Vamos a utilizar correos de prueba para verificar como se comporta, para eso vamos a utilizar algun otro corpus.

Este codigo sigue la misma logica para el corpus 1 ya que debemos obtener la misma estructura de clave:valor.

In [82]:
from nltk.util import bigrams

ruta = "/content/datasets/email/plaintext/corpus3.zip" # este es un corpus 3 de correos en inglish

data = []

with zipfile.ZipFile(ruta, mode="r") as corpus:
    for file in corpus.namelist():
        with corpus.open(file) as f:
            text = f.read().decode('ISO-8859-1')  # Decodificar el texto
            if text.strip():  # Verificar si el texto no está vacío

                tokens = nltk.regexp_tokenize(text,pattern) # tokenizar y realizar filtro con expresión regulares
                # Filtrar palabras que no están en stop words y que tienen más de 4 caracteres
                long_words = [palabra for palabra in tokens if palabra.lower() not in stop_words and len(palabra) > 2] # aqui estamos sacando las palabras que no estan en stopwords y ademas con longitud mayor a dos

                bigramas = list(bigrams(long_words))
                bandera = 3
                fil_bigramas = [bigrams for bigrams in bigramas if len(bigrams[0]) > bandera and len(bigrams[1]) > bandera]
                fdistBigrams = nltk.FreqDist(fil_bigramas)
                top_bigrams = fdistBigrams.most_common(5)

                # # Etiquetar el archivo según su clase (ham o spam)
                # #label = 'ham' if 'ham' in file else 'spam'
                label = 'True' if 'ham' in file else 'False'

                data.append((top_bigrams,label))
                random.shuffle(data)


fset_corpus_test = [(documento_atributo_bigram(top_bigrams),
         clase) for top_bigrams,clase in zip(df["top_bigrams"],df["Clase"])]

random.shuffle(fset_corpus_test)



In [83]:
fset_corpus_test[:2] # aqui teneos los primeros 2 datos

[({"contains((('timing', 'audit'), 2))": (('timing', 'audit'), 2),
   "contains((('corp', 'enron'), 2))": (('corp', 'enron'), 2),
   "contains((('enron', 'enron'), 2))": (('enron', 'enron'), 2),
   "contains((('Subject', 'timing'), 1))": (('Subject', 'timing'), 1),
   "contains((('audit', 'forwarding'), 1))": (('audit', 'forwarding'), 1)},
  'True'),
 ({"contains((('Subject', 'perfect'), 1))": (('Subject', 'perfect'), 1),
   "contains((('perfect', 'visual'), 1))": (('perfect', 'visual'), 1),
   "contains((('visual', 'solution'), 1))": (('visual', 'solution'), 1),
   "contains((('solution', 'business'), 1))": (('solution', 'business'), 1),
   "contains((('business', 'working'), 1))": (('business', 'working'), 1)},
  'False')]

In [84]:
clasif.classify(fset_corpus_test[5][0]) # aqui utilizamos un solo dato son etiqueta para prediccir el resultado.

'False'

In [85]:
ac(clasif,fset_corpus_test) # metrica de accuracy utilzando el clasif creado y el corpus 3 o 2

0.9969082125603864

<font size="+2"> Podemos apreciar que con los datos del corpus 2 y corpus 3 obtenemos resultados de presición favorables. tener presente que la accuracy es es la proporción de instancias clasificadas correctamente, tanto positivas como negativas, sobre el total de instancias.</font>